In [1]:
using ITensors

In [8]:
s = siteinds("S=1/2", 4)
mps = randomMPS(ComplexF64, s; linkdims=4);

In [9]:
struct PState
    pstate::MPS
    label::Int
end

In [4]:
function complex_feature_map(x::Float64)
    s1 = exp(1im * (3π/2) * x) * cospi(0.5 * x)
    s2 = exp(-1im * (2π/2) * x) * sinpi(0.5 * x)
    return [s1, s2]
end

complex_feature_map (generic function with 1 method)

In [10]:
function generate_training_data(samples_per_class::Int)

    class_A_samples = zeros(samples_per_class, 4)
    class_B_samples = ones(samples_per_class, 4)
    all_samples = vcat(class_A_samples, class_B_samples)
    all_labels = Int.(vcat(zeros(size(class_A_samples)[1]), ones(size(class_B_samples)[1])))

    return all_samples, all_labels

end

generate_training_data (generic function with 1 method)

In [7]:
generate_training_data(2)

([0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0; 1.0 1.0 1.0 1.0; 1.0 1.0 1.0 1.0], [0, 0, 1, 1])

In [11]:
function sample_to_product_state(sample::Vector, site_inds::Vector{Index{Int64}})
    n_sites = length(site_inds)
    product_state = MPS(ComplexF64, site_inds; linkdims=1)
    for j=1:n_sites
        T = ITensor(site_inds[j])
        zero_state, one_state = complex_feature_map(sample[j])
        T[1] = zero_state
        T[2] = one_state
        product_state[j] = T 
    end
    return product_state
end

sample_to_product_state (generic function with 1 method)

In [12]:
function dataset_to_product_state(dataset::Matrix, labels::Vector, sites::Vector{Index{Int64}})

    all_product_states = Vector{PState}(undef, size(dataset)[1])
    for p=1:length(all_product_states)
        sample_pstate = sample_to_product_state(dataset[p, :], sites)
        sample_label = labels[p]
        product_state = PState(sample_pstate, sample_label)
        all_product_states[p] = product_state
    end

    return all_product_states

end

dataset_to_product_state (generic function with 1 method)

In [13]:
function loss_and_grad(B::ITensor, mps::MPS, product_state::PState, lid, rid)
    # for a single sample
    ps = product_state.pstate
    phi_tilde = ps[lid] * ps[rid] * mps[3] * ps[3] * mps[4] * ps[4]
    yhat = B * phi_tilde
    y = product_state.label
    diff_sq = (norm(yhat[] - y))^2
    loss = 0.5 * diff_sq

    dP = yhat[] - y
    grad = 0.5 * dP * conj(phi_tilde)

    return loss, grad
end

loss_and_grad (generic function with 1 method)

In [14]:
all_samples, all_labels = generate_training_data(100)
all_pstates = dataset_to_product_state(all_samples, all_labels, s);

In [15]:
BT = mps[1] * mps[2]

ITensor ord=3 (dim=2|id=983|"S=1/2,Site,n=1") (dim=2|id=673|"S=1/2,Site,n=2") (dim=4|id=887|"Link,l=2")
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}

In [16]:
function update(n_iters, BT, lr=0.8)
    BT_new = BT
    for iter in 1:n_iters
        loss_total, grad_total = loss_and_grad(BT_new, mps, all_pstates[1], 1, 2)
        for ps in 2:200
            loss_val, grad_val = loss_and_grad(BT_new, mps, all_pstates[ps], 1, 2)
            loss_total += loss_val
            grad_total += grad_val
        end
        loss_final = loss_total / 200
        grad_final = grad_total ./ 200
        println("Loss: $loss_final")
        BT_new = BT_new - lr * grad_final
    end
    return BT_new
end

update (generic function with 2 methods)

In [17]:
BT_new = update(100, BT)

Loss: 0.4686150135926879
Loss: 0.29991360869932016
Loss: 0.19194470956756482
Loss: 0.1228446141232415
Loss: 0.0786205530388746
Loss: 0.05031715394487967
Loss: 0.03220297852472303
Loss: 0.020609906255822718
Loss: 0.013190340003726517
Loss: 0.008441817602384986
Loss: 0.005402763265526399
Loss: 0.003457768489936893
Loss: 0.002212971833559619
Loss: 0.001416301973478151
Loss: 0.0009064332630260168
Loss: 0.0005801172883366526
Loss: 0.0003712750645354556
Loss: 0.00023761604130269315
Loss: 0.000152074266433724
Loss: 9.73275305175821e-5
Loss: 6.228961953125293e-5
Loss: 3.9865356500002044e-5
Loss: 2.551382816000168e-5
Loss: 1.6328850022400578e-5
Loss: 1.0450464014336635e-5
Loss: 6.6882969691755504e-6
Loss: 4.280510060272275e-6
Loss: 2.7395264385741894e-6
Loss: 1.7532969206875293e-6
Loss: 1.1221100292401085e-6
Loss: 7.181504187135787e-7
Loss: 4.5961626797673417e-7
Loss: 2.9415441150511077e-7
Loss: 1.8825882336324366e-7
Loss: 1.2048564695248293e-7
Loss: 7.711081404963025e-8
Loss: 4.935092099173517

ITensor ord=3 (dim=2|id=983|"S=1/2,Site,n=1") (dim=2|id=673|"S=1/2,Site,n=2") (dim=4|id=887|"Link,l=2")
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}

In [18]:
test_ps = all_pstates[150].pstate

MPS
[1] ((dim=2|id=983|"S=1/2,Site,n=1"),)
[2] ((dim=2|id=673|"S=1/2,Site,n=2"),)
[3] ((dim=2|id=812|"S=1/2,Site,n=3"),)
[4] ((dim=2|id=496|"S=1/2,Site,n=4"),)


In [19]:
yhat = BT_new * test_ps[1] * test_ps[2] * mps[3] * test_ps[3] * mps[4] * test_ps[4]

ITensor ord=0
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}

In [20]:
abs(yhat[])

0.9999999997339204

# Have a look at sites 2-3

In [21]:
function loss_and_grad23(B::ITensor, mps::MPS, product_state::PState, lid, rid)
    # for a single sample
    ps = product_state.pstate
    phi_tilde = ps[lid] * ps[rid] * mps[1] * ps[1] * mps[4] * ps[4]
    yhat = B * phi_tilde
    y = product_state.label
    diff_sq = (norm(yhat[] - y))^2
    loss = 0.5 * diff_sq

    dP = yhat[] - y
    grad = 0.5 * dP * conj(phi_tilde)

    return loss, grad
end

loss_and_grad23 (generic function with 1 method)

In [22]:
B23 = mps[2] * mps[3]

ITensor ord=4 (dim=4|id=1|"Link,l=1") (dim=2|id=673|"S=1/2,Site,n=2") (dim=2|id=812|"S=1/2,Site,n=3") (dim=2|id=995|"Link,l=3")
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}

In [23]:
function update23(n_iters, BT, lr=0.8)
    BT_new = BT
    for iter in 1:n_iters
        loss_total, grad_total = loss_and_grad23(BT_new, mps, all_pstates[1], 2, 3)
        for ps in 2:200
            loss_val, grad_val = loss_and_grad23(BT_new, mps, all_pstates[ps], 2, 3)
            loss_total += loss_val
            grad_total += grad_val
        end
        loss_final = loss_total / 200
        grad_final = grad_total ./ 200
        println("Loss: $loss_final")
        BT_new = BT_new - lr * grad_final
    end
    return BT_new
end

update23 (generic function with 2 methods)

In [24]:
B23_new = update23(100, B23, 1.5)

Loss: 0.4686150135926879
Loss: 0.2878799983763153
Loss: 0.1773533493953199
Loss: 0.10961677549455923
Loss: 0.06800118469604398
Loss: 0.042360652541819824
Loss: 0.026511218360854606
Loss: 0.01667770277755604
Loss: 0.010551132486291043
Loss: 0.006716232883120261
Loss: 0.004303344644623091
Loss: 0.002776539301853299
Loss: 0.0018044627376459328
Loss: 0.001181478900129664
Loss: 0.0007794334112527626
Loss: 0.000518082784625414
Loss: 0.00034692054987900606
Loss: 0.00023397532710690716
Loss: 0.0001588833056595182
Loss: 0.00010858795691792185
Loss: 7.46592729711903e-5
Loss: 5.161467193134318e-5
Loss: 3.5861773090656766e-5
Loss: 2.5028897049719284e-5
Loss: 1.7538456892557524e-5
Loss: 1.233331063728162e-5
Loss: 8.699997082325732e-6
Loss: 6.153719215497112e-6
Loss: 4.362944326018045e-6
Loss: 3.0996007206447505e-6
Loss: 2.205933609282028e-6
Loss: 1.5722846990558653e-6
Loss: 1.1220889126326176e-6
Loss: 8.016752733876612e-7
Loss: 5.732893507728316e-7
Loss: 4.1029140516602096e-7
Loss: 2.93833873284664

ITensor ord=4 (dim=4|id=1|"Link,l=1") (dim=2|id=673|"S=1/2,Site,n=2") (dim=2|id=812|"S=1/2,Site,n=3") (dim=2|id=995|"Link,l=3")
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}

# Sites 3-4

In [25]:
function loss_and_grad34(B::ITensor, mps::MPS, product_state::PState, lid, rid)
    # for a single sample
    ps = product_state.pstate
    phi_tilde = ps[lid] * ps[rid] * mps[1] * ps[1] * mps[2] * ps[2]
    yhat = B * phi_tilde
    y = product_state.label
    diff_sq = (norm(yhat[] - y))^2
    loss = 0.5 * diff_sq

    dP = yhat[] - y
    grad = 0.5 * dP * conj(phi_tilde)

    return loss, grad
end

loss_and_grad34 (generic function with 1 method)

In [26]:
function update34(n_iters, BT, lr=0.8)
    BT_new = BT
    for iter in 1:n_iters
        loss_total, grad_total = loss_and_grad34(BT_new, mps, all_pstates[1], 3, 4)
        for ps in 2:200
            loss_val, grad_val = loss_and_grad34(BT_new, mps, all_pstates[ps], 3, 4)
            loss_total += loss_val
            grad_total += grad_val
        end
        loss_final = loss_total / 200
        grad_final = grad_total ./ 200
        println("Loss: $loss_final")
        BT_new = BT_new - lr * grad_final
    end
    return BT_new
end

update34 (generic function with 2 methods)

In [27]:
B34 = mps[3] * mps[4]

ITensor ord=3 (dim=4|id=887|"Link,l=2") (dim=2|id=812|"S=1/2,Site,n=3") (dim=2|id=496|"S=1/2,Site,n=4")
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}

In [28]:
B34_new = update34(100, B34, 0.8)

Loss: 0.4686150135926879
Loss: 0.4101368798193873
Loss: 0.3589650599876388
Loss: 0.31418570173934285
Loss: 0.27499936607255754
Loss: 0.24070669044928295
Loss: 0.21069584982101952
Loss: 0.18443158994875933
Loss: 0.1614456357252012
Loss: 0.14132830197871452
Loss: 0.12372115589829251
Loss: 0.10831059915755384
Loss: 0.09482225437578018
Loss: 0.08301605503396954
Loss: 0.07268195062527165
Loss: 0.06363614989057026
Loss: 0.055717834671050494
Loss: 0.04878628537513345
Loss: 0.04271836645977568
Loss: 0.037406326799452556
Loss: 0.03275587547680599
Loss: 0.02868449847909148
Loss: 0.025119986113387914
Loss: 0.021999144739108266
Loss: 0.019266669726852446
Loss: 0.016874159447762473
Loss: 0.014779252629405265
Loss: 0.012944873628437109
Loss: 0.011338572106770308
Loss: 0.00993194529152703
Loss: 0.008700132480226414
Loss: 0.00762137274805671
Loss: 0.006676617947052055
Loss: 0.005849194077904517
Loss: 0.0051245049818209685
Loss: 0.004489773057868257
Loss: 0.00393381237426647
Loss: 0.0034468301220106235

ITensor ord=3 (dim=4|id=887|"Link,l=2") (dim=2|id=812|"S=1/2,Site,n=3") (dim=2|id=496|"S=1/2,Site,n=4")
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}

In [29]:
function loss_12(B::ITensor, mps::MPS, product_state::PState, lid, rid)
    # for a single sample
    ps = product_state.pstate
    phi_tilde = ps[lid] * ps[rid] * mps[3] * ps[3] * mps[4] * ps[4]
    yhat = B * phi_tilde
    y = product_state.label
    diff_sq = (norm(yhat[] - y))^2
    loss = 0.5 * diff_sq

    return loss
end

loss_12 (generic function with 1 method)

In [30]:
B12 = mps[1] * mps[2]

ITensor ord=3 (dim=2|id=983|"S=1/2,Site,n=1") (dim=2|id=673|"S=1/2,Site,n=2") (dim=4|id=887|"Link,l=2")
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}

In [31]:
@show B12

B12 = ITensor ord=3
Dim 1: (dim=2|id=983|"S=1/2,Site,n=1")
Dim 2: (dim=2|id=673|"S=1/2,Site,n=2")
Dim 3: (dim=4|id=887|"Link,l=2")
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}
 2×2×4
[:, :, 1] =
 -0.11118921306805446 - 0.06440469388498564im   0.04162649817185639 - 0.2459497501908999im
 -0.10871016598010287 + 0.032387891138593604im  -0.1749794428231066 - 0.4105610731648804im

[:, :, 2] =
 -0.21691433242528474 - 0.2821632892686133im    0.07245590063260503 - 0.0006858625919498427im
 -0.37762035808709177 + 0.041308107385137356im  0.04576149918644166 - 0.1982232056338062im

[:, :, 3] =
  -0.1745745683778211 - 0.10051979534319651im  -0.1595874661820152 + 0.12810094125181729im
 -0.06295320250722447 + 0.29724787715746487im  0.21987210317120323 + 0.0542382181871879im

[:, :, 4] =
   0.1823768346906096 - 0.2615707039229628im   -0.035392863673559156 - 0.09919149879220243im
 -0.09495938604913497 - 0.09796149266682272im    0.11824008307812976 + 0.14206156411580875im


ITensor ord=3 (dim=2|id=983|"S=1/2,Site,n=1") (dim=2|id=673|"S=1/2,Site,n=2") (dim=4|id=887|"Link,l=2")
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}

In [32]:
loss_12(B12, mps, all_pstates[1], 1, 2)

0.08392264099781312

In [33]:
using Optim
using LinearAlgebra

In [34]:
i = Index(2, "i")
j = Index(2, "j")

(dim=2|id=346|"j")

In [35]:
B = randomITensor(i, j)

ITensor ord=2 (dim=2|id=877|"i") (dim=2|id=346|"j")
NDTensors.Dense{Float64, Vector{Float64}}

In [36]:
@show B

B = ITensor ord=2
Dim 1: (dim=2|id=877|"i")
Dim 2: (dim=2|id=346|"j")
NDTensors.Dense{Float64, Vector{Float64}}
 2×2
 -1.137682000681354   1.582096521343938
  1.9496332581006184  0.8820009989133135


ITensor ord=2 (dim=2|id=877|"i") (dim=2|id=346|"j")
NDTensors.Dense{Float64, Vector{Float64}}

In [37]:
T = 5.0

5.0

In [38]:
function cost_function(A_elems)
    A = ITensor(i, j)
    A[1,1] = A_elems[1]
    A[1,2] = A_elems[2]
    A[2,1] = A_elems[3]
    A[2,2] = A_elems[4]

    C = A * B
    C_val = scalar(C)
    cost = (C_val - T)^2
    return cost
end


cost_function (generic function with 1 method)

In [39]:
initial_guess = [0.1, 0.1, 0.1, 0.1]

4-element Vector{Float64}:
 0.1
 0.1
 0.1
 0.1

In [40]:
result = optimize(cost_function, initial_guess, ConjugateGradient())

 * Status: success

 * Candidate solution
    Final objective value:     2.154708e-22

 * Found with
    Algorithm:     Conjugate Gradient

 * Convergence measures
    |x - x'|               = 1.09e+00 ≰ 0.0e+00
    |x - x'|/|x'|          = 9.16e-01 ≰ 0.0e+00
    |f(x) - f(x')|         = 2.18e+01 ≰ 0.0e+00
    |f(x) - f(x')|/|f(x')| = 1.01e+23 ≰ 0.0e+00
    |g(x)|                 = 5.72e-11 ≤ 1.0e-08

 * Work counters
    Seconds run:   0  (vs limit Inf)
    Iterations:    1
    f(x) calls:    3
    ∇f(x) calls:   3


In [41]:
Optim.minimizer(result)

4-element Vector{Float64}:
 -0.5346084854817587
  0.9825066026469497
  1.187521652316433
  0.5919874954434333

In [44]:
final = ITensor(i, j)

ITensor ord=2 (dim=2|id=877|"i") (dim=2|id=346|"j")
NDTensors.EmptyStorage{NDTensors.EmptyNumber, NDTensors.Dense{NDTensors.EmptyNumber, Vector{NDTensors.EmptyNumber}}}

In [45]:
final[1, 1] = -0.5346084854817587
final[1,2] = 0.9825066026469497
final[2,1] = 1.187521652316433
final[2,2] = 0.5919874954434333

0.5919874954434333

In [46]:
@show final * B

final * B = ITensor ord=0
NDTensors.Dense{Float64, Vector{Float64}}
 0-dimensional
4.999999999985321


ITensor ord=0
NDTensors.Dense{Float64, Vector{Float64}}

In [588]:
i = Index(2, "i")
j = Index(2, "j")

(dim=2|id=868|"j")

In [589]:
B = randomITensor(ComplexF64, i, j)

ITensor ord=2 (dim=2|id=248|"i") (dim=2|id=868|"j")
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}

In [590]:
@show B

B = ITensor ord=2
Dim 1: (dim=2|id=248|"i")
Dim 2: (dim=2|id=868|"j")
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}
 2×2
  0.6516366826050329 - 0.3387672264355376im  -0.3955437836195299 + 1.1126374155072278im
 -0.8214268778237057 + 0.9092882513835908im  -0.6943601670492309 + 0.058139028743604465im


ITensor ord=2 (dim=2|id=248|"i") (dim=2|id=868|"j")
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}

In [591]:
T = 5.0 + 3.0im

5.0 + 3.0im